# 3. Joining & visualizing socioeconomic & vegetation data.

## Introduction

**Purpose:** Join the socioeconomic data (from `1-census-data-cleaning.ipynb`) and the vegetation density data (from `2-tract-vegetation.ipynb`) to identify and visualize census tract-level socioeconomic factors correlated to greenspace access.

**Contents:** The notebook shows:
* {contents TBD upon completing analysis}

**Conclusions:** Key conclusions & outputs are:
* {conclusions & outputs TBD upon completion}

## Import required libraries

In [1]:
# For manipulating tabular data
import pandas as pd

## Join data

Socioeconomic data is stored in the `acs2020.db` SQLite database, with `demographics`, `housing`, and `economics` tables. Vegetation density data is stored in the `vegetation.db` SQLite database in the `ndvi` table.